# Linear Regression 1

Regression is an attemp to find the equation that best fits the data, and be able forecast out a specific value.
A linear regression is as the name state, we use a linear equation.

In this intro will the tutorial use the data from quandl
Quandl is a platform for financial, economic, and alternative data that serves investment professionals.
and it as a great api available in python! (so, cool I will try it out more later)

to install Quandl api

pip install quandl

import necessary lib

In [1]:
import pandas as pd
import quandl

In [4]:
#getting data that grabs the stock price for Alphabet (previously Google), with the ticker of GOOGL:
df = quandl.get("WIKI/GOOGL")

print(df.head())

              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2004-08-19  100.01  104.06   95.96  100.335  44659000.0          0.0   
2004-08-20  101.01  109.08  100.50  108.310  22834300.0          0.0   
2004-08-23  110.76  113.48  109.05  109.400  18256100.0          0.0   
2004-08-24  111.24  111.60  103.57  104.870  15247300.0          0.0   
2004-08-25  104.76  108.00  103.88  106.000   9188600.0          0.0   

            Split Ratio  Adj. Open  Adj. High   Adj. Low  Adj. Close  \
Date                                                                   
2004-08-19          1.0  50.159839  52.191109  48.128568   50.322842   
2004-08-20          1.0  50.661387  54.708881  50.405597   54.322689   
2004-08-23          1.0  55.551482  56.915693  54.693835   54.869377   
2004-08-24          1.0  55.792225  55.972783  51.945350   52.597363   
2004-08-25          1.0  52.542193  54.167209  52.100830   53.1

In [5]:
# try to understand more about our data frame 
type(df)

pandas.core.frame.DataFrame

## filter

That a bit too mucn of an information we have, to see will filter out the data an choose only features we are interst in

In [6]:
df = df[['Adj. Open',  'Adj. High',  'Adj. Low',  'Adj. Close', 'Adj. Volume']]
print(df)

              Adj. Open    Adj. High     Adj. Low   Adj. Close  Adj. Volume
Date                                                                       
2004-08-19    50.159839    52.191109    48.128568    50.322842   44659000.0
2004-08-20    50.661387    54.708881    50.405597    54.322689   22834300.0
2004-08-23    55.551482    56.915693    54.693835    54.869377   18256100.0
2004-08-24    55.792225    55.972783    51.945350    52.597363   15247300.0
2004-08-25    52.542193    54.167209    52.100830    53.164113    9188600.0
2004-08-26    52.637487    54.142132    52.492038    54.122070    7094800.0
2004-08-27    54.217364    54.478169    53.008633    53.239345    6211700.0
2004-08-30    52.802998    52.908323    51.162935    51.162935    5196700.0
2004-08-31    51.318415    52.015567    51.238167    51.343492    4917800.0
2004-09-01    51.509003    51.644421    49.989312    50.280210    9138200.0
2004-09-02    49.698414    51.343492    49.623182    50.912161   15118600.0
2004-09-03  

## Processing Data

Quote from tutorial:
"Not all of the data you have is useful, and sometimes you need to do further manipulation on your data to make it even more valuable before feeding it through a machine learning algorithm"

In [7]:
# high low percentage
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Low']) / df['Adj. Low'] * 100.0
# percentage chage on each day
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0
df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]
print(df.head())

            Adj. Close    HL_PCT  PCT_change  Adj. Volume
Date                                                     
2004-08-19   50.322842  8.441017    0.324968   44659000.0
2004-08-20   54.322689  8.537313    7.227007   22834300.0
2004-08-23   54.869377  4.062357   -1.227880   18256100.0
2004-08-24   52.597363  7.753210   -5.726357   15247300.0
2004-08-25   53.164113  3.966115    1.183658    9188600.0


In [9]:
# save it to use later
df.to_pickle('wiki-GOOGL-df.pkl')

In [4]:
# load the data frame from pickle
df = pd.read_pickle('wiki-GOOGL-df.pkl')
print(df.head())

            Adj. Close    HL_PCT  PCT_change  Adj. Volume
Date                                                     
2004-08-19   50.322842  8.441017    0.324968   44659000.0
2004-08-20   54.322689  8.537313    7.227007   22834300.0
2004-08-23   54.869377  4.062357   -1.227880   18256100.0
2004-08-24   52.597363  7.753210   -5.726357   15247300.0
2004-08-25   53.164113  3.966115    1.183658    9188600.0


In [6]:
# library need from doing machine learning's regression problem
import quandl, math
import numpy as np
import pandas as pd
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression

C:\Users\Deemarc\AppData\Local\Continuum\miniconda3\envs\carnd-term1\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


 forecasting out the price, our label, the thing we're hoping to predict, is actually the future price.

In [10]:
#predicting the next closing price
forecast_col = 'Adj. Close'

# for missing data 'NaN' we replace it -99999 which will be treat as  outlier 
df.fillna(value=-99999, inplace=True)

#forcasting length: 1% of the data length
forecast_out = int(math.ceil(0.01 * len(df)))

In [12]:
print(forecast_out)

34


In [18]:
df['label'] = df[forecast_col].shift(-forecast_out)

In [21]:
#drop any data row that still have NaN
df.dropna(inplace=True)

In [28]:
X = np.array(df.drop(['label'],1))
y = np.array(df['label'])
print(X.shape)
print(y.shape)
print(X[1:10])

(3356, 4)
(3356,)
[[  5.43226889e+01   8.53731343e+00   7.22700723e+00   2.28343000e+07]
 [  5.48693765e+01   4.06235672e+00  -1.22788010e+00   1.82561000e+07]
 [  5.25973630e+01   7.75321039e+00  -5.72635743e+00   1.52473000e+07]
 [  5.31641125e+01   3.96611475e+00   1.18365788e+00   9.18860000e+06]
 [  5.41220696e+01   3.14351233e+00   2.82039066e+00   7.09480000e+06]
 [  5.32393448e+01   2.77225849e+00  -1.80388529e+00   6.21170000e+06]
 [  5.11629351e+01   3.41143025e+00  -3.10600304e+00   5.19670000e+06]
 [  5.13434924e+01   1.51722788e+00   4.88663018e-02   4.91780000e+06]
 [  5.02802102e+01   3.31092606e+00  -2.38558909e+00   9.13820000e+06]]


In [29]:
#normalize the data
X = preprocessing.scale(X)
print(X[1:10])

[[-1.41810563  4.38757624  4.82702292  1.79774234]
 [-1.41582954  1.31020692 -0.8023334   1.24526328]
 [-1.42528886  3.84835881 -3.79746873  0.8821732 ]
 [-1.42292925  1.24402257  0.80329493  0.15103325]
 [-1.41894089  0.67832962  1.89304948 -0.10163825]
 [-1.42261603  0.42302321 -1.18584395 -0.20820726]
 [-1.43126097  0.86257328 -2.05280817 -0.33069344]
 [-1.43050924 -0.44004496  0.0477383  -0.36434999]
 [-1.43493612  0.79345786 -1.57314869  0.14495118]]


In [41]:
# split the data from traning and testing, and also shuffle your data
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)
# use Support Vector Regression as a classifier
clf = svm.SVR()
clf.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [42]:
# test it out !
confidence = clf.score(X_test, y_test)
print(confidence)

0.827120119845


In [43]:
#try another classifier
clf2 = LinearRegression(n_jobs=-1) # n_jobs=-1 to use all available thread
clf2.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [44]:
# test it out !
confidence = clf2.score(X_test, y_test)
print(confidence)

0.976478941805


In [46]:
#try out some other kernel
for k in ['linear','poly','rbf','sigmoid']:
    clf = svm.SVR(kernel=k)
    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)
    print(k,confidence)

linear 0.975192675228
poly 0.656920050119
rbf 0.827120119845
sigmoid 0.899801188385
